In [1]:
import sys
sys.path.append("/Users/nicolasaunai/Documents/code/phare/phare/pharein")
import pharein as ph

In [2]:
ph.globals.sim = None

# user initialization script

In [3]:
#!/usr/bin/env python

import pharein
from pharein import Simulation
from pharein import MaxwellianFluidModel
from pharein import ElectromagDiagnostics
from pharein import FluidDiagnostics
from pharein import ParticleDiagnostics

In [4]:

#------------------------------------
#     configure the simulation
#------------------------------------

Simulation(
    time_step_nbr=1000,                   # number of time steps (not specified if time_step and final_time provided)
    final_time=1.,                        # simulation final time (not specified if time_step and time_step_nbr given)
    boundary_types="periodic",            # boundary condition, string or tuple, length == len(cell) == len(dl)
    cells=80,                             # integer or tuple length == dimension
    dl=0.1,                               # mesh size of the root level, float or tuple
    path='test5'                          # directory where INI file and diagnostics directories will be
    # time_step = 0.005,                  # simulation time step (not specified if time_step_nbr and final_time given)
    # domain_size = 8.,                   # float or tuple, not specified if dl and cells are
    # interp_order = 1,                   # interpolation order, [default = 1] can be 1, 2, 3 or 4
    # layout = "yee",                     # grid layout, [default="yee"]
    # origin = 0.,                        # position of the origin of the domain, float or tuple (length = dimension)
    # particle_pusher = "modified_boris", # particle pusher method, [default = "modified_boris"]
    # refined_particle_nbr = 2,           # number of refined particle a particle is split into [default : ]
    # diag_export_format = 'ascii',       # export format of the diagnostics [default = 'ascii']
    # refinement = {"level":[0,1],        # AMR parameters
    #                "extent_ratio":[0.4, 0.6],
    #                "refinement_iterations":[0, 3]},

) # end Simulation





Need to test boxes are OK (no overlap, in level range, etc.)


In [5]:

import numpy as np

def n(x):
    x0 = 5.
    return 1./np.cosh(x-x0)**2

def bx(x):
    x0=5.
    return np.tanh(x-x0)


MaxwellianFluidModel(bx=bx,
                     protons={"density":n},
                     background={})





In [6]:
ElectromagDiagnostics(
    name="ElectromagDiagnostics1",
    diag_type="E",                  # available : ("E", "B")
    write_every=10,
    compute_every=5,
    start_teration=0,
    last_iteration=990,
    path = 'ElectromagDiagnostics1'   # where output files will be written, [default: name]
) 

In [7]:
FluidDiagnostics(
    name="FluidDiagnostics1",       # name of the diagnostics
    diag_type="density",            # choose in (rho_s, flux_s)
    write_every=10,                 # write on disk every x iterations
    compute_every=5,                # compute diagnostics every x iterations ( x <= write_every)
    start_iteration=0,              # iteration at which diag is enabled
    last_iteration=990,             # iteration at which diag is turned off
    population_name="protons"       # name of the population for which the diagnostics is made
  #,path = 'FluidDiagnostics1'      # where output files will be written, [default: name]
)

In [8]:
FluidDiagnostics(
    name="FluidDiagnostics3",
    diag_type="bulk_velocity",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="background"
)

In [9]:
FluidDiagnostics(
    name="FluidDiagnostics2",
    diag_type="density",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="all"
)

In [16]:
pharein.globals.sim.diagnostics

In [15]:
FluidDiagnostics(
    name="FluidDiagnostics4",
    diag_type="flux",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="background"
)

Error: diagnostics FluidDiagnostics4 already registered


In [17]:
ElectromagDiagnostics(
    name="ElectromagDiagnostics2",
    diag_type="B",
    write_every=10,
    compute_every=5,
    start_teration=0,
    last_iteration=990
)

In [18]:
ParticleDiagnostics(
        name = "ParticleDiagnostics1",
        compute_every=10,
        write_every=10,
        start_iteration=0,
        last_iteration=90,
        diag_type="space_box",          # choose particles within a spatial box
        extent=(2., 4.),                # extent of the box
        population_name="protons"
        )

In [19]:
ph.globals.sim.model.model_dict

{'model': 'model',
 'model_name': 'custom',
 'bx': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'by': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'bz': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ex': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ey': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ez': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'protons': {'charge': 1.0,
  'mass': 1.0,
  'density': <function __main__.n(x)>,
  'vx': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
  'vy': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
  'vz': <function pharein.maxwellian_fluid_model.Maxwell

In [20]:
d = ph.globals.sim.model.model_dict["protons"]["density"]

In [21]:
d(2)

0.009866037165440192

In [22]:
ph.globals.sim.model.model_dict['bx']

<function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>

In [23]:
for diag in ph.globals.sim.diagnostics:
    print("simulation/diagnostics/name/" + diag.name)
    print("simulation/diagnostics/type/"  + diag.category)
    print("simulation/diagnostics/subtype/"  + diag.diag_type)
    print("simulation/diagnostics/compute_every/" + str(diag.compute_every))
    print("simulation/diagnostics/write_every/" + str(diag.write_every))
    print("simulation/diagnostics/start_iteration/" + str(diag.start_iteration))    
    print("simulation/diagnostics/last_iteration/" + str(diag.last_iteration))  
    if (diag.category in ["FluidDiagnostics", "ParticleDiagnostics"]):
        print("simulation/diagnostics/population/" + str(diag.population_name))
    print("")
    

simulation/diagnostics/name/ElectromagDiagnostics1
simulation/diagnostics/type/electromag
simulation/diagnostics/subtype/EM_E
simulation/diagnostics/compute_every/5
simulation/diagnostics/write_every/10
simulation/diagnostics/start_iteration/0
simulation/diagnostics/last_iteration/990

simulation/diagnostics/name/FluidDiagnostics1
simulation/diagnostics/type/FluidDiagnostics
simulation/diagnostics/subtype/density
simulation/diagnostics/compute_every/5
simulation/diagnostics/write_every/10
simulation/diagnostics/start_iteration/0
simulation/diagnostics/last_iteration/990
simulation/diagnostics/population/protons

simulation/diagnostics/name/FluidDiagnostics3
simulation/diagnostics/type/FluidDiagnostics
simulation/diagnostics/subtype/bulk_velocity
simulation/diagnostics/compute_every/5
simulation/diagnostics/write_every/10
simulation/diagnostics/start_iteration/0
simulation/diagnostics/last_iteration/990
simulation/diagnostics/population/background

simulation/diagnostics/name/FluidDiagn